## This  is a notebook

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_absolute_error

In [2]:
properties = pd.read_csv('../data/properties_2016.csv')
train_data = pd.read_csv('../data/train_2016_v2.csv')
template_submission = pd.read_csv('../data/sample_submission.csv')

C:\Users\x003752\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Preprocessing

- merge data frames
- remove crappy features (missing > .75)
- make sure the types make sense


we have data of houses that have been sold, those have a logerror. This logerror is our train_y. Split data set on logerror / no logerror. The other is test?



In [3]:
merged_df = pd.merge(train_data, properties,on='parcelid')

In [4]:
temp_df = merged_df[['bathroomcnt', 'bedroomcnt', 'longitude', 'latitude','logerror']].dropna()
properties_filtered = properties[['bathroomcnt', 'bedroomcnt', 'longitude', 'latitude']]
properties_filtered = properties_filtered.fillna(properties_filtered.mean())
x_train = temp_df[['bathroomcnt', 'bedroomcnt', 'longitude', 'latitude']]
y_train = temp_df.logerror
x_submission = properties_filtered

In [5]:
#impute grouped on unitcnt, buildingqualitytypeid, buildingclasstypeid, architecturalstyletypeid

## Feature engineering

In [6]:
feature_engineering = Imputer(strategy="median", axis = 0) 


## Model

In [7]:
## knn en lasso ?
neigbors = KNeighborsRegressor(n_neighbors=5, weights= 'distance',n_jobs=-1)



## Pipeline

In [8]:
pipe = Pipeline([('feature_engineering', feature_engineering),
                 ('normalize', Normalizer()),
                ('regressor', neigbors)
                 ]
)

In [9]:
# Fit model
pipe.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('feature_engineering', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('normalize', Normalizer(copy=True, norm='l2')), ('regressor', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='distance'))])

In [10]:
out_submission = pipe.predict(x_submission)

In [11]:
out_formatted = template_submission
for col in out_formatted.columns.values:
    if col == 'ParcelId':
        pass
    else:
        out_formatted[col] = out_submission
out_formatted.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,-0.016223,-0.016223,-0.016223,-0.016223,-0.016223,-0.016223
1,10759547,0.027647,0.027647,0.027647,0.027647,0.027647,0.027647
2,10843547,-0.013980,-0.013980,-0.013980,-0.013980,-0.013980,-0.013980
3,10859147,0.007505,0.007505,0.007505,0.007505,0.007505,0.007505
4,10879947,0.018970,0.018970,0.018970,0.018970,0.018970,0.018970


In [37]:
out_formatted.to_csv('../data/submission_RMH.gz', index=False, compression = "gzip")

In [12]:
out_formatted.to_csv('../data/submission_RMH', index=False)